In [4]:
from google.colab import files
uploaded = files.upload()   # choose sales_data.csv

Saving sales_data.csv to sales_data.csv


In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('SalesData').getOrCreate()

df=spark.read.format('csv').option('header','true').option('inferSchema','true').load('sales_data.csv')
df.show()

+-------+----------+---------------+------+--------+------+------+-----------+
|OrderID| OrderDate|       Category|Region|Quantity| Sales|Profit|SubCategory|
+-------+----------+---------------+------+--------+------+------+-----------+
|   1001|2023-04-13|Office Supplies| South|       8|205.35|175.55|    Storage|
|   1002|2023-12-15|     Technology|  West|       2|421.53|249.33|Accessories|
|   1003|2023-09-28|      Furniture|  West|       9|970.94|154.52|  Bookcases|
|   1004|2023-04-17|Office Supplies|  East|       7|295.23|204.45|    Binders|
|   1005|2023-03-13|     Technology|  West|       9| 673.9|-35.97|Accessories|
|   1006|2023-07-08|Office Supplies|  East|       6|358.93| 84.62|    Storage|
|   1007|2023-01-21|      Furniture| South|       6| 784.8|-96.27|  Bookcases|
|   1008|2023-04-13|      Furniture| South|       4|174.33| -1.33|     Tables|
|   1009|2023-05-02|      Furniture| North|       2|971.33|190.58|  Bookcases|
|   1010|2023-08-03|Office Supplies| South|       7|

In [12]:
from pyspark.sql.functions import *
df=df.toDF(*[col_name.strip().lower().replace(" ","_").replace("-","_").replace(",","_") for col_name in df.columns])

df=df.withColumn('quantity',col('quantity').cast('int'))\
     .withColumn('sales',col('sales').cast('double'))\
     .withColumn('profit',col('profit').cast('double'))

df_cleaned=df.dropna().dropDuplicates()

df_cleaned.show()
df_cleaned.printSchema()
df_cleaned.write.mode("overwrite").csv("cleaned_sales_data",header=True)


+-------+----------+---------------+------+--------+------+------+-----------+
|orderid| orderdate|       category|region|quantity| sales|profit|subcategory|
+-------+----------+---------------+------+--------+------+------+-----------+
|   1140|2023-12-18|      Furniture| South|       2|930.68|165.52|  Bookcases|
|   1478|2023-04-10|      Furniture| North|       6|487.51|-73.42|     Chairs|
|   1075|2023-12-17|Office Supplies| South|       8| 78.87|299.34|    Binders|
|   1398|2023-03-04|Office Supplies|  West|       4|465.72|102.51|    Binders|
|   1047|2023-06-16|      Furniture|  East|       7| 54.69|135.28|  Bookcases|
|   1096|2023-12-11|      Furniture|  West|       1|684.21|276.27|  Bookcases|
|   1229|2023-11-21|      Furniture| North|       9|272.03| 10.76|  Bookcases|
|   1283|2023-04-23|     Technology| North|       7|923.42| 40.95|    Laptops|
|   1385|2023-11-17|      Furniture|  East|       9|192.55|142.84|  Bookcases|
|   1394|2023-12-08|Office Supplies| North|       1|

In [13]:
!zip -r cleaned_sales_data.zip cleaned_sales_data

  adding: cleaned_sales_data/ (stored 0%)
  adding: cleaned_sales_data/._SUCCESS.crc (stored 0%)
  adding: cleaned_sales_data/_SUCCESS (stored 0%)
  adding: cleaned_sales_data/.part-00000-77c4b06b-130d-4ac5-8ce6-f707ce9a99e6-c000.csv.crc (stored 0%)
  adding: cleaned_sales_data/part-00000-77c4b06b-130d-4ac5-8ce6-f707ce9a99e6-c000.csv (deflated 71%)


In [14]:
from google.colab import files
files.download("cleaned_sales_data.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>